# Initialize

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
from pandas import Series, DataFrame
import pandas as pd

In [ ]:
import uuid

In [ ]:
str(uuid.uuid4())

In [14]:
# driver = webdriver.Chrome('/Users/jungwoosong/Documents/python/chromedriver')
driver = webdriver.Firefox(executable_path='/Users/jungwoosong/Documents/python/geckodriver')

In [15]:
driver.set_window_position("0","0")
driver.set_window_size("1280","800")

# Categories

###### Business & Finance

In [ ]:
category = 'Business & Finance'

In [ ]:
cur_category = category

In [16]:
driver.get('https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1502813450&rnid=13727922011')

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div.a-row.a-spacing-none.scx-truncate-medium.s-list-title-short > a")
# results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div.a-row.a-spacing-none.scx-truncate-medium.s-list-title-medium > a")
results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")

###### Get skills links

In [ ]:
hrefs = list()
for i in range(0,len(results)):
    hrefs.append(results[i]['href'])

## Enter link, get skill data

In [ ]:
driver.get(hrefs[0])
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

스킬명|카테고리명|저자명|레이팅|레이팅퍼센트_5|레이팅퍼센트_4|레이팅퍼센트_3|레이팅퍼센트_2|레이팅퍼센트_1|리뷰수|설명|커맨드|Invocation|skill_id(스킬하나가 고유한 id를 갖도록 임의로 부여, incremental)|timestamp
Primary key: skill_id / foreign key:스킬명 / row는 스킬당 하나 

### Get raw html data

In [ ]:
name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
ratings_percentage_raw = soup.select("#histogramTable > tbody")
review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
description_raw = soup.select("#a2s-description > span")
commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
invocation_raw = soup.select("#a2s-skill-details > ul > li")
# skill id
# timestamp

#### Check html data

In [ ]:
print(name_raw)
print(category)
print(author_raw)
print(rating_raw)
print(ratings_percentage_raw)
print(review_count_raw)
print(description_raw)
print(commands_raw)
print(invocation_raw)

## Set Skill dictionary

In [ ]:
skill = {}
skill['category'] = cur_category
skill['title'] = name_raw[0].get_text().strip()
skill['author'] = author_raw[0].get_text().strip()
skill['rating'] = rating_raw[0].get_text()
ratings_percentages = list()
for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
    if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
        ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
skill['ratingPercentage_5'] = ratings_percentages[0]
skill['ratingPercentage_4'] = ratings_percentages[1]
skill['ratingPercentage_3'] = ratings_percentages[2]
skill['ratingPercentage_2'] = ratings_percentages[3]
skill['ratingPercentage_1'] = ratings_percentages[4]
skill['reviewNum'] = review_count_raw[0].get_text().strip()
skill['description'] = description_raw[0].get_text()
commands = list()
for i in range(0,len(commands_raw)):
    commands.append(commands_raw[i].get_text().strip())
skill['commands'] = commands 
skill['invocation'] = invocation_raw[1].span.span.get_text() #없을수도있음
skill['timestamp'] = time.time()

In [ ]:
cur_skill_Title = skill['title']

In [ ]:
skill

## Enter link, get skill review data

In [ ]:
reviews_link = 'https://www.amazon.com'+soup.select("#dp-summary-see-all-reviews")[0]['href']

In [ ]:
driver.get(reviews_link)
driver.find_element_by_css_selector(".reviewer-type-select").click() #파폭(맥북스크린)
# driver.find_element_by_css_selector("#a-autoid-5-announce").click() #작은화면일때(맥북스크린)

In [ ]:
all_reviewers_flag = driver.find_element_by_css_selector("#reviewer-type-dropdown > option").text

## Change option 'Verified' to 'All reviers'

In [ ]:
if(all_reviewers_flag == 'All reviewers'):
    print('yes, it is all reviewers')
    driver.find_element_by_css_selector("#reviewer-type-dropdown_0").click()
else:
    print('wrong filter_not All reviewers')
    #error: end loop

## Toggle all comments' subComment

In [ ]:
comments_subComment_toggle_list = driver.find_elements_by_css_selector("div.a-row.a-spacing-top-small.review-comments > div > a") #comments_subComment

In [ ]:
comments_subComment_toggle_list

In [ ]:
for button in comments_subComment_toggle_list:
    print(button)
    button.location_once_scrolled_into_view
    element = WebDriverWait(driver, 10).until(
        EC.visibility_of(button)
    )
    element.click() #click all subcomments toggle

## Get skill review data & Set reviews list

### 1 page scrap

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reviews = list()
comments = list()
for i, review_data in enumerate(soup.select("#cm_cr-review_list")[0].children):
    if(review_data['class'] != ['a-section', 'review']): #review가 아니면 탈출, 마지막에 페이지 모듈도 children에 있어서 조건문 담
        break
    review = {}
    review['skill_title'] = cur_skill_Title
    review['title'] = review_data.div.find_all("a", class_="a-size-base a-link-normal review-title a-color-base a-text-bold")[0].get_text()
    review['author'] = review_data.div.find_all("a", class_="a-size-base a-link-normal author")[0].get_text() #review_author
    review['date'] = review_data.div.find_all("span", class_="a-size-base a-color-secondary review-date")[0].get_text() #review date
    review['rating'] = review_data.div.a['title'] 
    review['content'] = review_data.div.find_all("span", class_="a-size-base review-text")[0].get_text() #review_content
    if(review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span is not None): # 추천이 있으면
        review['vote'] = review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span.get_text().strip() #review_votes, strip()은 앞 뒤 공백제거
    else:
        review['vote'] = '0'
    comments_data = review_data.find_all("div", class_ = "a-section a-spacing-none review-comments")[0]
    if(comments_data.div is not None):
        print("there are comments")        
        for i, comment_data in enumerate(comments_data):
            if(comment_data['class'] == ['a-section', 'a-spacing-small', 'a-spacing-top-small', 'review-comment']):
                comment = {}   
                comment['review_title'] = review['title']
                comment['author'] = comment_data.find(attrs={"data-hook": "review-author"}).get_text() #subcomment author
                comment['date'] = comment_data.find("span", class_="a-color-tertiary comment-time-stamp").get_text() #subcomment date
                comment['content'] = comment_data.find("span", class_="review-comment-text").get_text() #subcomment content
                comments.append(comment)
    else:
        print("no comments")
    review['commentNum'] = review_data.div.find_all("span", class_="review-comment-total aok-hidden")[0].get_text() #comment_num
    review['timestamp'] = time.time()
    reviews.append(review)

In [ ]:
reviews

In [ ]:
comments

In [ ]:
len(reviews)

In [ ]:
len(comments)

## Move to next page

In [ ]:
# selenium ver
# driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class')
# driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'
if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
    print('This is not a last page')
    driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
else:
    print('This is a last page')
    print('Move to next Skill')    
# element = driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a") #current page's next page
# if(element is None)
# element.click()

In [ ]:
soup.find_all("#cm_cr-pagination_bar > ul > li.a-last")

In [ ]:
# bs ver
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
if(soup.find_all("#cm_cr-pagination_bar > ul > li.a-last")):
    if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
        print('This is not a last page')
        driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
    else:
        print('This is a last page')
        print('Move to next Skill')    


## All review page scrap - loop

### YesReview_Comments

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reviews = list()
comments = list()
while(True):
# while(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
    time.sleep(2) # wait 2secs
    comments_subComment_toggle_list = driver.find_elements_by_css_selector("div.a-row.a-spacing-top-small.review-comments > div > a") #comments_subComment
    for button in comments_subComment_toggle_list:
        print(button)
        button.location_once_scrolled_into_view
        element = WebDriverWait(driver, 10).until(
            EC.visibility_of(button)
        )
        element.click() #click all subcomments toggle
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    for i, review_data in enumerate(soup.select("#cm_cr-review_list")[0].children):
        if(review_data['class'] != ['a-section', 'review']): #review가 아니면 탈출, 마지막에 페이지 모듈도 children에 있어서 조건문 담
            break
        review = {}
        review['skill_title'] = cur_skill_Title
        review['title'] = review_data.div.find_all("a", class_="a-size-base a-link-normal review-title a-color-base a-text-bold")[0].get_text()
        review['author'] = review_data.div.find_all("a", class_="a-size-base a-link-normal author")[0].get_text() #review_author
        review['date'] = review_data.div.find_all("span", class_="a-size-base a-color-secondary review-date")[0].get_text() #review date
        review['rating'] = review_data.div.a['title'] 
        review['content'] = review_data.div.find_all("span", class_="a-size-base review-text")[0].get_text() #review_content
        if(review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span is not None): # 추천이 있으면
            review['vote'] = review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span.get_text().strip() #review_votes, strip()은 앞 뒤 공백제거
        else:
            review['vote'] = '0'
        comments_data = review_data.find_all("div", class_ = "a-section a-spacing-none review-comments")[0]
        if(comments_data.div is not None): #comment가 있다면
            print("there are comments")        
            for i, comment_data in enumerate(comments_data):
                if(comment_data['class'] == ['a-section', 'a-spacing-small', 'a-spacing-top-small', 'review-comment']):
                    comment = {}   
                    comment['review_title'] = review['title']
                    comment['author'] = comment_data.find(attrs={"data-hook": "review-author"}).get_text() #subcomment author
                    comment['date'] = comment_data.find("span", class_="a-color-tertiary comment-time-stamp").get_text() #subcomment date
                    comment['content'] = comment_data.find("span", class_="review-comment-text").get_text() #subcomment content
                    comments.append(comment)
        else:
            print("no comments")
        review['commentNum'] = review_data.div.find_all("span", class_="review-comment-total aok-hidden")[0].get_text() #comment_num
        review['timestamp'] = time.time()
        reviews.append(review)
#     if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') == 'a-disabled a-last'):
#         print('This is a last page')
#         break
#     else:
#         print('This is not a last page')
#         driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
    if(soup.find_all("#cm_cr-pagination_bar > ul > li.a-last") != []):
        if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
            print('This is not a last page')
            driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
    else:
        print('This is a last page')
        print('Move to next Skill')
        break

In [ ]:
len(reviews)

In [ ]:
reviews

In [ ]:
comments

### No Review_Comments

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reviews = list()
comments = list()
while(True):
# while(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
    time.sleep(2) # wait 2secs
#     comments_subComment_toggle_list = driver.find_elements_by_css_selector("div.a-row.a-spacing-top-small.review-comments > div > a") #comments_subComment
#     for button in comments_subComment_toggle_list:
#         print(button)
#         button.location_once_scrolled_into_view
#         element = WebDriverWait(driver, 10).until(
#             EC.visibility_of(button)
#         )
#         element.click() #click all subcomments toggle
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    for i, review_data in enumerate(soup.select("#cm_cr-review_list")[0].children):
        if(review_data['class'] != ['a-section', 'review']): #review가 아니면 탈출, 마지막에 페이지 모듈도 children에 있어서 조건문 담
            break
        review = {}
        review['skill_title'] = cur_skill_Title
        review['title'] = review_data.div.find_all("a", class_="a-size-base a-link-normal review-title a-color-base a-text-bold")[0].get_text()
        review['author'] = review_data.div.find_all("a", class_="a-size-base a-link-normal author")[0].get_text() #review_author
        review['date'] = review_data.div.find_all("span", class_="a-size-base a-color-secondary review-date")[0].get_text() #review date
        review['rating'] = review_data.div.a['title'] 
        review['content'] = review_data.div.find_all("span", class_="a-size-base review-text")[0].get_text() #review_content
        if(review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span is not None): # 추천이 있으면
            review['vote'] = review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span.get_text().strip() #review_votes, strip()은 앞 뒤 공백제거
        else:
            review['vote'] = '0'
        comments_data = review_data.find_all("div", class_ = "a-section a-spacing-none review-comments")[0]
#         if(comments_data.div is not None): #comment가 있다면
#             print("there are comments")        
#             for i, comment_data in enumerate(comments_data):
#                 if(comment_data['class'] == ['a-section', 'a-spacing-small', 'a-spacing-top-small', 'review-comment']):
#                     comment = {}   
#                     comment['review_title'] = review['title']
#                     comment['author'] = comment_data.find(attrs={"data-hook": "review-author"}).get_text() #subcomment author
#                     comment['date'] = comment_data.find("span", class_="a-color-tertiary comment-time-stamp").get_text() #subcomment date
#                     comment['content'] = comment_data.find("span", class_="review-comment-text").get_text() #subcomment content
#                     comments.append(comment)
#         else:
#             print("no comments")
        review['commentNum'] = review_data.div.find_all("span", class_="review-comment-total aok-hidden")[0].get_text() #comment_num
        review['timestamp'] = time.time()
        reviews.append(review)
    if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') == 'a-disabled a-last'):
        print('This is a last page')
        break
    else:
        print('This is not a last page')
        driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()

# Scrap one page skills

In [ ]:
category = 'Business & Finance'
cur_category = category

In [ ]:
driver.get('https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1502813450&rnid=13727922011')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# newest arrivals to check bugs
driver.get('https://www.amazon.com/s/ref=lp_14284819011_st?rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&qid=1503552728&sort=date-desc-rank')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
hrefs = list()
for i in range(0,len(results)):
    hrefs.append(results[i]['href'])

In [ ]:
#business & finance last page
driver.get('https://www.amazon.com/s/ref=lp_14284819011_pg_2?rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&page=25&ie=UTF8&qid=1503556312')
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
hrefs = list()
for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
    hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))

## Enter link, get skill data

In [ ]:
skills = list()
for link in hrefs:
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
    author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
    rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
    ratings_percentage_raw = soup.select("#histogramTable > tbody")
    review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
    description_raw = soup.select("#a2s-description > span")
    commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
    invocation_raw = soup.select("#a2s-skill-details > ul > li")
    # skill id
    # timestamp
    skill = {}
    skill['category'] = cur_category
    skill['title'] = name_raw[0].get_text().strip()
    skill['author'] = author_raw[0].get_text().strip()
    if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
        print("Rating exist")
        print("rating_raw = " + rating_raw[0].get_text())
        skill['rating'] = rating_raw[0].get_text()
        ratings_percentages = list()
        for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
            if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
        skill['ratingPercentage_5'] = ratings_percentages[0]
        skill['ratingPercentage_4'] = ratings_percentages[1]
        skill['ratingPercentage_3'] = ratings_percentages[2]
        skill['ratingPercentage_2'] = ratings_percentages[3]
        skill['ratingPercentage_1'] = ratings_percentages[4]        
    else:
        print("Rating doesn't exist")
        skill['rating'] = '0'
        skill['ratingPercentage_5'] = '0'
        skill['ratingPercentage_4'] = '0'
        skill['ratingPercentage_3'] = '0'
        skill['ratingPercentage_2'] = '0'
        skill['ratingPercentage_1'] = '0'
    skill['reviewNum'] = review_count_raw[0].get_text().strip()
    skill['description'] = description_raw[0].get_text()
    commands = list()
    for i in range(0,len(commands_raw)):
        commands.append(commands_raw[i].get_text().strip())
    skill['commands'] = commands 
    for i, inv in enumerate(invocation_raw):
        if("Invocation Name" in inv.span.get_text()):
            print("invocation exist")
            skill['invocation'] = invocation_raw[i].span.span.get_text()
            break
        else:
            print('invocation does not exist')
            skill['invocation'] = 'No_Invocation'
    skill['timestamp'] = time.time()
    skills.append(skill)
    cur_skill_Title = skill['title']

In [ ]:
skills

In [ ]:
skills_df = DataFrame(skills)

In [ ]:
skills_df

## Move to next skills page

In [ ]:
for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
#     print(child.get_attribute("class"))
    if(child.get_attribute("class") == 'pagnRA'):
        print('This is not a last page')
        print(child.get_attribute("class"))
        print(child)
        child.find_element_by_id('pagnNextString').click()
#         break
    elif(child.get_attribute("class") == 'pagnRA1'):
        print('This is a last page')

In [ ]:
driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*")

In [ ]:
pd.concat([skills_df,skills_df])

# Scrap several skills page

In [ ]:
driver.current_url

In [ ]:
driver.get(cur_page)
temp = []
for i in range(len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
    temp.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))

In [ ]:
len(skills)

## Scrap only skill data

In [ ]:
cur_page = 'https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1503558721&rnid=13727922011'
category = 'Business & Finance'
cur_category = category
driver.get(cur_page)
skills = list()
exitFlag = False
# for i in range(3):
while(exitFlag == False):
    time.sleep(2)
    cur_page = driver.current_url
    hrefs = list()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
    for i in range(0,len(results)):
        hrefs.append(results[i]['href'])
#     for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
#         hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))
#     time.sleep(2)
    for link in hrefs:
        time.sleep(1)
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
        author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
        rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
        ratings_percentage_raw = soup.select("#histogramTable > tbody")
        review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
        description_raw = soup.select("#a2s-description > span")
        commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
        invocation_raw = soup.select("#a2s-skill-details > ul > li")
        # skill id
        # timestamp
        skill = {}
        skill['category'] = cur_category
        skill['title'] = name_raw[0].get_text().strip()
        skill['author'] = author_raw[0].get_text().strip()
        if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
            print("Rating exist")
            print("rating_raw = " + rating_raw[0].get_text())
            skill['rating'] = rating_raw[0].get_text()
            ratings_percentages = list()
            for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
                if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                    ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
            skill['ratingPercentage_5'] = ratings_percentages[0]
            skill['ratingPercentage_4'] = ratings_percentages[1]
            skill['ratingPercentage_3'] = ratings_percentages[2]
            skill['ratingPercentage_2'] = ratings_percentages[3]
            skill['ratingPercentage_1'] = ratings_percentages[4]        
        else:
            print("Rating doesn't exist")
            skill['rating'] = '0'
            skill['ratingPercentage_5'] = '0'
            skill['ratingPercentage_4'] = '0'
            skill['ratingPercentage_3'] = '0'
            skill['ratingPercentage_2'] = '0'
            skill['ratingPercentage_1'] = '0'
        skill['reviewNum'] = review_count_raw[0].get_text().strip()
        skill['description'] = description_raw[0].get_text()
        commands = list()
        for i in range(0,len(commands_raw)):
            commands.append(commands_raw[i].get_text().strip())
        skill['commands'] = commands 
        for i, inv in enumerate(invocation_raw):
            if("Invocation Name" in inv.span.get_text()):
                print("invocation exist")
                skill['invocation'] = invocation_raw[i].span.span.get_text()
                break
            else:
                print('invocation does not exist')
                skill['invocation'] = 'No_Invocation'
        skill['timestamp'] = time.time()
        skills.append(skill)
        cur_skill_Title = skill['title']
    driver.get(cur_page)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break

## Scrap skill data & review data & comment data

In [30]:
cur_page = 'https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1503558721&rnid=13727922011'
category = 'Business & Finance'
cur_category = category
driver.get(cur_page)
skills = list()
reviews = list()
comments = list()
exitFlag = False
for i in range(2):
# while(exitFlag == False):
    time.sleep(2)
    cur_page = driver.current_url
    hrefs = list()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
#     for i in range(0,len(results)):
    for i in range(0,len(results)-8): #테스트용도로 페이지마다 8개씩만
        hrefs.append(results[i]['href'])
#     for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
#         hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))
#     time.sleep(2)
    for link in hrefs:
        time.sleep(1)
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
        author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
        rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
        ratings_percentage_raw = soup.select("#histogramTable > tbody")
        review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
        description_raw = soup.select("#a2s-description > span")
        commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
        invocation_raw = soup.select("#a2s-skill-details > ul > li")
        # skill id
        # timestamp
        skill = {}
        skill['category'] = cur_category
        skill['title'] = name_raw[0].get_text().strip()
        skill['author'] = author_raw[0].get_text().strip()
        if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
            print("Rating exist")
            print("rating_raw = " + rating_raw[0].get_text())
            skill['rating'] = rating_raw[0].get_text()
            ratings_percentages = list()
            for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
                if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                    ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
            skill['ratingPercentage_5'] = ratings_percentages[0]
            skill['ratingPercentage_4'] = ratings_percentages[1]
            skill['ratingPercentage_3'] = ratings_percentages[2]
            skill['ratingPercentage_2'] = ratings_percentages[3]
            skill['ratingPercentage_1'] = ratings_percentages[4]        
        else:
            print("Rating doesn't exist")
            skill['rating'] = '0'
            skill['ratingPercentage_5'] = '0'
            skill['ratingPercentage_4'] = '0'
            skill['ratingPercentage_3'] = '0'
            skill['ratingPercentage_2'] = '0'
            skill['ratingPercentage_1'] = '0'
        skill['reviewNum'] = review_count_raw[0].get_text().strip()
        skill['description'] = description_raw[0].get_text()
        commands = list()
        for i in range(0,len(commands_raw)):
            commands.append(commands_raw[i].get_text().strip())
        skill['commands'] = commands 
        for i, inv in enumerate(invocation_raw):
            if("Invocation Name" in inv.span.get_text()):
                print("invocation exist")
                skill['invocation'] = invocation_raw[i].span.span.get_text()
                break
            else:
                print('invocation does not exist')
                skill['invocation'] = 'No_Invocation'
        skill['timestamp'] = time.time()
        skills.append(skill)
        cur_skill_Title = skill['title']
        if(soup.select("#dp-summary-see-all-reviews") != []): #review가 있을 때
            reviews_link = 'https://www.amazon.com'+soup.select("#dp-summary-see-all-reviews")[0]['href'] #review link가 없는 경우도 있다 # coin watch처럼
            driver.get(reviews_link)
            driver.find_element_by_css_selector(".reviewer-type-select").click() #파폭(맥북스크린)
            all_reviewers_flag = driver.find_element_by_css_selector("#reviewer-type-dropdown > option").text
            if(all_reviewers_flag == 'All reviewers'):
                print('yes, it is all reviewers')
                driver.find_element_by_css_selector("#reviewer-type-dropdown_0").click()
            else:
                print('wrong filter_not All reviewers')
                break
            while(True):
                time.sleep(2) # wait 2secs
                comments_subComment_toggle_list = driver.find_elements_by_css_selector("div.a-row.a-spacing-top-small.review-comments > div > a") #comments_subComment
                for button in comments_subComment_toggle_list:
                    print(button)
                    button.location_once_scrolled_into_view
                    element = WebDriverWait(driver, 10).until(
                        EC.visibility_of(button)
                    )
                    element.click() #click all subcomments toggle
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                for i, review_data in enumerate(soup.select("#cm_cr-review_list")[0].children):
                    if(review_data['class'] != ['a-section', 'review']): #review가 아니면 탈출, 마지막에 페이지 모듈도 children에 있어서 조건문 담
                        break
                    review = {}
                    review['skill_title'] = cur_skill_Title
                    review['title'] = review_data.div.find_all("a", class_="a-size-base a-link-normal review-title a-color-base a-text-bold")[0].get_text()
                    review['author'] = review_data.div.find_all("a", class_="a-size-base a-link-normal author")[0].get_text() #review_author
                    review['date'] = review_data.div.find_all("span", class_="a-size-base a-color-secondary review-date")[0].get_text() #review date
                    review['rating'] = review_data.div.a['title'] 
                    review['content'] = review_data.div.find_all("span", class_="a-size-base review-text")[0].get_text() #review_content
                    if(review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span is not None): # 추천이 있으면
                        review['vote'] = review_data.div.find_all("span", class_="cr-vote-buttons cr-vote-component")[0].span.span.get_text().strip() #review_votes, strip()은 앞 뒤 공백제거
                    else:
                        review['vote'] = '0'
                    comments_data = review_data.find_all("div", class_ = "a-section a-spacing-none review-comments")[0]
                    if(comments_data.div is not None): #comment가 있다면
                        print("there are comments")        
                        for i, comment_data in enumerate(comments_data):
                            if(comment_data['class'] == ['a-section', 'a-spacing-small', 'a-spacing-top-small', 'review-comment']):
                                comment = {}   
                                comment['review_title'] = review['title']
                                comment['author'] = comment_data.find(attrs={"data-hook": "review-author"}).get_text() #subcomment author
                                comment['date'] = comment_data.find("span", class_="a-color-tertiary comment-time-stamp").get_text() #subcomment date
                                comment['content'] = comment_data.find("span", class_="review-comment-text").get_text() #subcomment content
                                comments.append(comment)
                    else:
                        print("no comments")
                    review['commentNum'] = review_data.div.find_all("span", class_="review-comment-total aok-hidden")[0].get_text() #comment_num
                    review['timestamp'] = time.time()
                    reviews.append(review)
                if(soup.select("#cm_cr-pagination_bar > ul > li.a-last") != []):
                    if(driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last").get_attribute('class') != 'a-disabled a-last'):
                        print('This is not a last page')
                        driver.find_element_by_css_selector("#cm_cr-pagination_bar > ul > li.a-last > a").click()
                    else:
                        print('This is a last page')
                        print('Move to next Skill')
                        break
                else:
                    print('There are no other pages')
                    break
        else: #review가 없을 때
            print('No review link')        
    driver.get(cur_page)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break

Rating exist
rating_raw = 3.5 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="7f3b2701-a897-ea44-ab3b-75af5514d9cb")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="7f3213ea-f710-9247-89e4-95416adbb28a")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="5fef313c-bd67-a648-bda3-1cd5f546faf1")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="16cd8d6b-87eb-c642-990d-51f94ae508ae")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="5c0ff7c3-a29d-0343-bd31-00df195f6232")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", elemen

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="01a50fbc-6611-764c-aa3a-e765905344ae")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="9a6c3233-ebf8-f94c-a3cd-e72053fa480b")>
no comments
no comments
no comments
there are comments
there are comments
there are comments
there are comments
there are comments
there are comments
no comments
This is not a last page
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="928fec4c-ff9d-984d-b967-6c813fbfbe29")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="9968b14a-3678-6446-ba0a-d7feab0b3681")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="f2e991ea-7a09-4d4a-8606-dd5352745b90")>
<selenium.webdriver.firefox.webelement.

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="d2251442-9fc0-214c-9e96-3db4699a24f0")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="8f8b95bf-c237-6c4a-a1a5-2e01073b9f1a")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="fa89361f-df93-cf49-89a2-8f4d0de2b44d")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="4fd20cb6-d450-9540-bcd4-54fe422ac103")>
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
no comments
This is not a last page
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="f15afa8f-e0f5-ee4c-9b9c-e60994dff2bc")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="e3088032-742d-914e-a5f6-af3988d0a6fe")>
no comments
no comments
no comments
no comments
no comments
This is a last page
Move to next Skill
Rating exist
rating_raw = 5.0 out of 5 stars
invocation does not exist
invocation exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="012aaa10-2402-a74c-8dc3-5ef5f80d984a")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="0bca31c5-6498-a84b-9ab0-26f21af3e664")>
no comments
no comments
There are no other pages
Rating exist
rating_raw = 4.0 out of 5 stars
invocation does not exist
invocation does not exist
yes, it is all reviewers
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="948714ba-eb08-b642-ae41-669f25f3cac5"

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="335568d3-da37-744e-89b7-ebd5666c0a89")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="7a453b9e-156c-e848-a40c-0b965362b3ce")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="7c937036-b3cf-ee4c-8cf9-cb9bff0f1f6f")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="23ec6fb8-5a63-ad45-b754-7a3891ee1a91")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="2e919cf0-b79a-fb44-8898-83c738664d22")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d57e4e3-0e27-964d-be9c-3921c6ca3501", element="2677cb8b-35fd-9f40-9dec-8698b5a66ded")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="1d5

In [35]:
len(skills)

16

In [37]:
DataFrame(skills)

,author,category,commands,description,invocation,rating,ratingPercentage_1,ratingPercentage_2,ratingPercentage_3,ratingPercentage_4,ratingPercentage_5,reviewNum,timestamp,title
0,"by Capital One Services, LLC",Business & Finance,"[""Alexa, Open Capital One"", ""Alexa, ask Capita...","Ask Capital One about your credit card, checki...",capital one,3.5 out of 5 stars,23%,4%,8%,12%,53%,145,1.503575e+09,Capital One
1,by Andy K.,Business & Finance,"[""Alexa, open bitcoin exchange"", ""Alexa, ask b...",Bitcoin can say the current exchange rate of b...,bitcoin exchange,5.0 out of 5 stars,0%,0%,0%,0%,100%,2,1.503575e+09,Bitcoin Exchange
2,by The Motley Fool,Business & Finance,"[""Alexa, what's my Flash Briefing?"", ""Alexa, w...",Helping the world invest better – one stock at...,No_Invocation,4.0 out of 5 stars,9%,0%,19%,5%,67%,21,1.503575e+09,Motley Fool Stock Watch
3,by wxrdaniel,Business & Finance,"[""Alexa, ask coin base how much is bitcoin?"", ...",Check Realtime bitcoin and ethereum price from...,coin base,4.1 out of 5 stars,0%,0%,33%,17%,50%,6,1.503575e+09,Coinbase Bitcoin/Ethereum Price
4,by Doug Devitre International Inc.,Business & Finance,"[""Alexa Ask Why Remax"", ""Why Join Remax"", ""Why...",This simple Alexa skill was architected to giv...,why remax,5.0 out of 5 stars,0%,0%,0%,0%,100%,4,1.503575e+09,Why RE/MAX
5,by YNAB,Business & Finance,"[""Alexa, ask Why Nab for the balance of Grocer...",Have Alexa help you gain total control of your...,why nab,3.6 out of 5 stars,22%,6%,9%,10%,53%,32,1.503575e+09,YNAB
6,by Sade Smith,Business & Finance,"[""Alexa, ask Coin Watch what is the price of b...",Coin Watch gives the current USD price of the ...,coin watch,0,0,0,0,0,0,0,1.503575e+09,Coin Watch
7,by Doug Devitre International Inc.,Business & Finance,"[""Alexa, open womens council"", ""why should I g...",Getting involved in the Women's Council of REA...,women's council,5.0 out of 5 stars,0%,0%,0%,0%,100%,2,1.503575e+09,Women's Council
8,by Safeco Insurance,Business & Finance,"[""Alexa, open Insurance Advisor"", ""Alexa, ask ...",Confused by insurance? Insurance Advisor by Sa...,insurance advisor,5.0 out of 5 stars,0%,0%,0%,0%,100%,2,1.503575e+09,Insurance Advisor
9,by Cold Winter Development,Business & Finance,"[""Alexa, ask litecoin price What is the price ...",Ask and receive the spot price of Litecoin bas...,litecoin price,0,0,0,0,0,0,0,1.503575e+09,Litecoin Price


In [38]:
DataFrame(reviews)

,author,commentNum,content,date,rating,skill_title,timestamp,title,vote
0,R. Seehoffer,1,The skill itself isn't bad--in fact I used it ...,"on January 10, 2017",3.0 out of 5 stars,Capital One,1.503575e+09,"Not a bad skill, but access pin is readable by...",27 people found this helpful.
1,Logan,1,This is an awesome feature. However I can't se...,"on December 1, 2016",4.0 out of 5 stars,Capital One,1.503575e+09,Does ALMOST Everything Perfect,14 people found this helpful.
2,Antonio,0,Very convenient to stay up-to-date information...,"on December 2, 2016",5.0 out of 5 stars,Capital One,1.503575e+09,"Great Skill, Very Useful",11 people found this helpful.
3,Dustin,0,Love capital one! They invest more in tech tha...,"on October 21, 2016",5.0 out of 5 stars,Capital One,1.503575e+09,Cool!,8 people found this helpful.
4,Tom Poole,0,"Technology done right! Frictionless, easy peas...","on December 3, 2016",5.0 out of 5 stars,Capital One,1.503575e+09,Awesome Skill!!!,7 people found this helpful.
5,Maria M. Graciano,1,Would be nice if you could restrict which of y...,"on November 29, 2016",4.0 out of 5 stars,Capital One,1.503575e+09,Four Stars,7 people found this helpful.
6,s@mw1se,4,This is amazing! I'm sure the functionality wi...,"on March 11, 2016",5.0 out of 5 stars,Capital One,1.503575e+09,"Alexa, you just made my day!",15 people found this helpful.
7,Mike303,1,"Extremely useful to check balance, the days sp...","on July 11, 2017",5.0 out of 5 stars,Capital One,1.503575e+09,Excellent!,3 people found this helpful.
8,Andrew E Wilkinson,1,A slight bug... when I request to make a paym...,"on October 21, 2016",3.0 out of 5 stars,Capital One,1.503575e+09,Promising overall! Noticed an NLU bug.,5 people found this helpful.
9,Amazon Customer,0,"May need a lil tweaking , only only to make pa...","on November 26, 2016",4.0 out of 5 stars,Capital One,1.503575e+09,Mulitiple card,10 people found this helpful.


In [34]:
len(reviews)

223

In [39]:
DataFrame(comments)

,author,content,date,review_title
0,Capital One,We're happy to hear you're liking the skill ov...,6 months ago,"Not a bad skill, but access pin is readable by..."
1,Amazon Customer,Hi Logan. We're excited that you're happy with...,7 months ago,Does ALMOST Everything Perfect
2,Ray S.,Yes !!!!! Absolutely because I share accounts ...,7 months ago,Four Stars
3,Andy Kim,Great comment,9 months ago,"Alexa, you just made my day!"
4,Andy Kim,Thanks!,9 months ago,"Alexa, you just made my day!"
5,John B.,Agreed,9 months ago,"Alexa, you just made my day!"
6,Andy Kim,Awesome review,9 months ago,"Alexa, you just made my day!"
7,Capital One,We're glad to be setting the bar for modern da...,28 days ago,Excellent!
8,Capital One,We'd like to ask that you give this another tr...,9 months ago,Promising overall! Noticed an NLU bug.
9,Capital One,Hey Bradley! We've got an awesome team hard at...,9 months ago,Worth Enabling


In [36]:
len(comments)

56

In [25]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [29]:
soup.select("#dp-summary-see-all-reviews") == []

True

In [27]:
soup.select("#dp-summary-see-all-reviews") #review link가 없는 경우도 있다

[]

In [ ]:
soup.

## Test Code

In [ ]:
cur_page = 'https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1503558721&rnid=13727922011'
category = 'Business & Finance'
cur_category = category
driver.get(cur_page)
skills = list()
hrefs_list = list()
results_list = list()
exitFlag = False
for i in range(3):
# while(exitFlag == False):
    time.sleep(1)
    cur_page = driver.current_url
    hrefs = list()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
    results_list.append(results)
    for i in range(0,len(results)):
        hrefs.append(results[i]['href'])
#     for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
#         hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))    
    hrefs_list.append(hrefs)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break
'''    for link in hrefs:
        time.sleep(1)
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
        author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
        rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
        ratings_percentage_raw = soup.select("#histogramTable > tbody")
        review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
        description_raw = soup.select("#a2s-description > span")
        commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
        invocation_raw = soup.select("#a2s-skill-details > ul > li")
        # skill id
        # timestamp
        skill = {}
        skill['category'] = cur_category
        skill['title'] = name_raw[0].get_text().strip()
        skill['author'] = author_raw[0].get_text().strip()
        if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
            print("Rating exist")
            print("rating_raw = " + rating_raw[0].get_text())
            skill['rating'] = rating_raw[0].get_text()
            ratings_percentages = list()
            for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
                if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                    ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
            skill['ratingPercentage_5'] = ratings_percentages[0]
            skill['ratingPercentage_4'] = ratings_percentages[1]
            skill['ratingPercentage_3'] = ratings_percentages[2]
            skill['ratingPercentage_2'] = ratings_percentages[3]
            skill['ratingPercentage_1'] = ratings_percentages[4]        
        else:
            print("Rating doesn't exist")
            skill['rating'] = '0'
            skill['ratingPercentage_5'] = '0'
            skill['ratingPercentage_4'] = '0'
            skill['ratingPercentage_3'] = '0'
            skill['ratingPercentage_2'] = '0'
            skill['ratingPercentage_1'] = '0'
        skill['reviewNum'] = review_count_raw[0].get_text().strip()
        skill['description'] = description_raw[0].get_text()
        commands = list()
        for i in range(0,len(commands_raw)):
            commands.append(commands_raw[i].get_text().strip())
        skill['commands'] = commands 
        for i, inv in enumerate(invocation_raw):
            if("Invocation Name" in inv.span.get_text()):
                print("invocation exist")
                skill['invocation'] = invocation_raw[i].span.span.get_text()
                break
            else:
                print('invocation does not exist')
                skill['invocation'] = 'No_Invocation'
        skill['timestamp'] = time.time()
        skills.append(skill)
        cur_skill_Title = skill['title']
    driver.get(cur_page)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break'''

In [ ]:
len(hrefs_list[2])

In [ ]:
len(results_list[2])

# Scrap several skills page with comments

In [ ]:
cur_page = 'https://www.amazon.com/s/ref=lp_13727921011_nr_n_0?fst=as%3Aoff&rh=n%3A13727921011%2Cn%3A%2113727922011%2Cn%3A14284819011&bbn=13727922011&ie=UTF8&qid=1503558721&rnid=13727922011'
category = 'Business & Finance'
cur_category = category
driver.get(cur_page)
skills = list()
exitFlag = False
for i in range(3):
# while(exitFlag == False):
    time.sleep(2)
    cur_page = driver.current_url
    hrefs = list()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.select("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")
    for i in range(0,len(results)):
        hrefs.append(results[i]['href'])
#     for i in range(0,len(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a"))):
#         hrefs.append(driver.find_elements_by_css_selector("div > div > div > div.a-fixed-left-grid-col.a-col-right > div.a-row.a-spacing-small > div > a")[i].get_attribute('href'))
#     time.sleep(2)
    for link in hrefs:
        time.sleep(1)
        driver.get(link)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        name_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > h1")
        author_raw = soup.select("#a2s-product-info > div.a-section.a-spacing-micro.a2s-title > span")
        rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span > a > span")
        ratings_percentage_raw = soup.select("#histogramTable > tbody")
        review_count_raw = soup.select("#a2s-product-info > div.a2s-pdd-reviews > span > a > span")
        description_raw = soup.select("#a2s-description > span")
        commands_raw = soup.select("#a2s-utterance-carousel > div > div > div > div > div > ol > li > div > div > span.a2s-utterance-text > em")
        invocation_raw = soup.select("#a2s-skill-details > ul > li")
        # skill id
        # timestamp
        skill = {}
        skill['category'] = cur_category
        skill['title'] = name_raw[0].get_text().strip()
        skill['author'] = author_raw[0].get_text().strip()
        if(soup.select("#reviewSummary > div.a-row.a-spacing-small")  != []):
            print("Rating exist")
            print("rating_raw = " + rating_raw[0].get_text())
            skill['rating'] = rating_raw[0].get_text()
            ratings_percentages = list()
            for i, rating_percentage in enumerate(ratings_percentage_raw[0].find_all("td", class_="aok-nowrap")):
                if((i % 2) != 0): #짝수로 나눠지지 않으면, rating의 퍼센트이니 ratings_percentages list에 넣어준다
                    ratings_percentages.append(rating_percentage.get_text()) #넣어지는 순서는 5, 4, 3, 2, 1 star
            skill['ratingPercentage_5'] = ratings_percentages[0]
            skill['ratingPercentage_4'] = ratings_percentages[1]
            skill['ratingPercentage_3'] = ratings_percentages[2]
            skill['ratingPercentage_2'] = ratings_percentages[3]
            skill['ratingPercentage_1'] = ratings_percentages[4]        
        else:
            print("Rating doesn't exist")
            skill['rating'] = '0'
            skill['ratingPercentage_5'] = '0'
            skill['ratingPercentage_4'] = '0'
            skill['ratingPercentage_3'] = '0'
            skill['ratingPercentage_2'] = '0'
            skill['ratingPercentage_1'] = '0'
        skill['reviewNum'] = review_count_raw[0].get_text().strip()
        skill['description'] = description_raw[0].get_text()
        commands = list()
        for i in range(0,len(commands_raw)):
            commands.append(commands_raw[i].get_text().strip())
        skill['commands'] = commands 
        for i, inv in enumerate(invocation_raw):
            if("Invocation Name" in inv.span.get_text()):
                print("invocation exist")
                skill['invocation'] = invocation_raw[i].span.span.get_text()
                break
            else:
                print('invocation does not exist')
                skill['invocation'] = 'No_Invocation'
        skill['timestamp'] = time.time()
        skills.append(skill)
        cur_skill_Title = skill['title']
    driver.get(cur_page)
    for child in driver.find_element_by_css_selector("#pagn").find_elements_by_css_selector("*"):
        if(child.get_attribute("class") == 'pagnRA'):
            exitFlag = False
            child.find_element_by_id('pagnNextString').click()
            print('Go to next page')
            break
        elif(child.get_attribute("class") == 'pagnRA1'):
            print('This is a last page')
            print('Crawling for one catrgory - Done')
            exitFlag = True
            break

# 버그들

#### 레이팅이 없으면 에러남 - 해결책, if문으로 조건 체크

###### 16개씩 정상적으로 긁다가 갑자기 12개씩 긁음

In [ ]:
skill

In [ ]:
driver.get(hrefs[0])
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small > span")

In [ ]:
soup.select("#reviewSummary") != []

In [ ]:
rating_raw = soup.select("#reviewSummary > div.a-row.a-spacing-small")

In [ ]:
rating_raw

In [ ]:
invocation_raw = soup.select("#a2s-skill-details > ul > li")


In [ ]:
invocation_raw[1].span.span.get_text()

In [ ]:
"Invocation Name" in invocation_raw[1].span.get_text()

In [ ]:
invocation_raw[1].span.get_text()

In [ ]:
for i, inv in enumerate(invocation_raw):
    if("Invocation Name" in inv.span.get_text()):
        print("invocation exist")
    else:
        print('invocation does not exist')

스킬명|카테고리명|저자명|레이팅|레이팅퍼센트_5|레이팅퍼센트_4|레이팅퍼센트_3|레이팅퍼센트_2|레이팅퍼센트_1|리뷰수|설명|커맨드|Invocation|skill_id(스킬하나가 고유한 id를 갖도록 임의로 부여, incremental)|timestamp
Primary key: skill_id / foreign key:스킬명 / row는 스킬당 하나 

In [ ]:
cur_skill_Title = skill['title']

# Make DataFrame

In [ ]:
skills_df = DataFrame(skills)

In [ ]:
skills

In [ ]:
skills_df

In [ ]:
skill_df['title']

In [ ]:
reviews_df = DataFrame(reviews)

In [ ]:
reviews_df

In [ ]:
comments_df = DataFrame(comments)

In [ ]:
comments_df

### Write to CSV

In [ ]:
skills_df

In [ ]:
skills_df.to_csv('./skills_business & Finance_skill&review.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [ ]:
skill_df.to_csv('./skills3.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [ ]:
reviews_df.to_csv('./reviews.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')

In [ ]:
comments_df.to_csv('./comments.csv',sep=',',na_rep='NaN',header=False, index=False, encoding='utf-8')